In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
test_results = pd.read_csv("/Users/codyschank/Dropbox/Insight/18B/data_challenge_2/test_results.csv")
user_table = pd.read_csv("/Users/codyschank/Dropbox/Insight/18B/data_challenge_2/user_table.csv")

## Cleaning

In [14]:
# I did this
test_results.loc[test_results.test == 0, 'price'] = 39.0
test_results.loc[test_results.test == 1, 'price'] = 59.0

In [15]:
# faster to do this
test_results['price'] = test_results['test'].map({0: 39.0, 1: 59.0})

In [16]:
test_results = test_results.drop(columns='Unnamed: 0')

In [17]:
test_results.head()

,user_id,timestamp,source,device,operative_system,test,price,converted
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39.0,0
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39.0,0
2,317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39.0,0
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59.0,0
4,820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39.0,0


In [18]:
# Cleaning of timestamp, which I didn't do any of
# is the first line necessary? What will to_datetime do with that?
test_results['timestamp'] = test_results['timestamp'].str.replace(':60',':00')
test_results['timestamp'] = test_results['timestamp'].str.replace('2815','2015') #One of the years was mistakenly labeled 2815 instead of 2015
test_results['timestamp'] = pd.to_datetime(test_results['timestamp'], format='%Y-%m-%d %H:%M:%S')
test_results.head()

,user_id,timestamp,source,device,operative_system,test,price,converted
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39.0,0
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39.0,0
2,317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39.0,0
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,iOS,1,59.0,0
4,820854,2015-05-24 11:04:40,ads_facebook,web,mac,0,39.0,0


## Stats

Not surprisingly there is a lower conversion rate for the higher price point but is there a statistically significant difference? To test this I used a two way proportion z score test since this is binary data and ultimately proportion data.

In [8]:
import statsmodels.stats.proportion as ssp

conversion_contigency_table = test_results.groupby(['price','converted'])['user_id'].count()
counts = [conversion_contigency_table[39][1], conversion_contigency_table[59][1]]

nobs = test_results.groupby('price')['user_id'].count().values

z_stat, p_value = ssp.proportions_ztest(count=counts, nobs=nobs, alternative='two-sided')

print(z_stat, p_value)

8.782770948335264 1.5949576477472644e-18


In [47]:
nobs

array([202727, 114073])

Clearly there is a significant decrease in conversion rate for the higher price point. So this means that in terms of revenue that it will depend on how much product is being sold and how many people are viewing the website.

Which you just multiply conversion rate by price, which I did too. __But is there a way to test the statistical significance of this comparison? Use another proportion z score test?__

## Could Company XYZ Have Found The Same Results in Less Time?

First I need to calculate the N needed to get a power >=.8 __0.8 seems to be standard? Why ratio 0.5?__

Remember, this below is just for overall conversion rate. __What if we wanted to think about conversion rates across the different user segements?__

http://www.statsmodels.org/dev/generated/statsmodels.stats.power.tt_ind_solve_power.html

https://stats.stackexchange.com/questions/59202/stats-relationship-between-alpha-and-beta

Conversion rate is low, so effect size is low?

In [9]:
import statsmodels.stats.power as sspower

effect_size = ssp.proportion_effectsize(counts[1]/nobs[1], counts[0]/nobs[0])
nobs1 = None #this is what we are observing
alpha = .05
power = .8 # this is beta
ratio = .5 # is this related to nobs?

n_group_39 = sspower.zt_ind_solve_power(effect_size = effect_size, nobs1=None, alpha=alpha, power=power, ratio = ratio)
total_n = n_group_39 * (1+ratio)
print('Total people needed: ' + str(total_n))

Total people needed: 32210.766487079418


In [41]:
dates = test_results['timestamp'].sort_values()
print()
print('Could have run test for: '+ str(dates.iloc[int(total_n)+1] - dates.iloc[0]))


Could have run test for: 11 days 01:45:21


In [39]:
dates = dates.dropna()
print('Total time test was run: '+ str(dates.iloc[-1] - dates.iloc[0]))

Total time test was run: 90 days 23:55:33
